## Hallucination Score Analysis

In this notebook, the hallucination scores of the methods are analyzed.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nqopen_scores = pd.read_csv('results\\togetherai_2_nqopen_with_ground_truths__mistralai_Mistral-7B-Instruct-v0-1.csv')
xsum_scores = pd.read_csv('results\\togetherai_2_xsum_with_ground_truths__mistralai_Mistral-7B-Instruct-v0-1.csv')


In [3]:
nqopen_scores.head()

,answer,prompt,llm_answer,llm_answer__1,llm_answer__2,llm_answer__3,llm_answer__4,llm_answer__5,selfcheck_nli_score,selfcheck_bert_score,fleek_score,lm_v_lm_score,lbhd_score,ground_truth
0,"['14 December 1972 UTC', 'December 1972']",when was the last time anyone was on the moon,"(["" ['"", 'The', ' last', ' time', ' anyone', '...","(None, None, None, ""['The last time anyone was...","(None, None, None, ""['the last crewed mission ...","(None, None, None, ""['the last person on the m...","(None, None, None, ""['Charlie Bates was the la...","(None, None, None, '[Apollo 18 was the last Ap...",0.810109,0.546060,0,0.0,"{'avg': 0.29954681108807507, 'normalized_produ...",1.0
1,"['Bobby Scott', 'Bob Russell']",who wrote he ain't heavy he's my brother lyrics,"(["" ['"", 'The', ' lyrics', ' to', ' ""', 'He', ...","(None, None, None, '[\'The lyrics to the song ...","(None, None, None, '[\'Derek and the Dominos w...","(None, None, None, '[\'The lyrics to ""He Ain\'...","(None, None, None, '[\'Paul McCartney wrote th...","(None, None, None, '[\'Bob Dylan wrote the ""He...",0.998452,0.455366,0,0.0,"{'avg': 0.39881583601590387, 'normalized_produ...",1.0
2,"['one', 'one season']",how many seasons of the bastard executioner ar...,"(["" ['"", 'The', ' Bast', 'ard', ' Execut', 'io...","(None, None, None, ""['There are two seasons of...","(None, None, None, ""['The Bastard Executioner ...","(None, None, None, ""['There is only one season...","(None, None, None, ""['The Bastard Executioner ...","(None, None, None, ""['the baste"")",0.479305,0.604782,0,0.0,"{'avg': 0.32746767171428576, 'normalized_produ...",0.0
3,['2017'],when did the eagles win last super bowl,"(["" ['"", 'The', ' Philadelphia', ' E', 'agles'...","(None, None, None, ""['the Eagles won the Super...","(None, None, None, ""['The Philadelphia Eagles ...","(None, None, None, ""['The Philadelphia Eagles ...","(None, None, None, ""['Philadelphia Eagles won ...","(None, None, None, ""['The Philadelphia Eagles ...",0.008632,0.363366,0,0.0,"{'avg': 0.40973532698984677, 'normalized_produ...",0.0
4,['South Carolina'],who won last year's ncaa women's basketball,"(["" ['"", 'U', 'Conn', ' won', ' last', ' year'...","(None, None, None, ""['UCLA won last year’s NCA...","(None, None, None, ""['Baylor won last year’s N...","(None, None, None, ""['Baylor won last year’s N...","(None, None, None, ""['UConn, University of Con...","(None, None, None, ""['The University of Connec...",0.599977,0.345637,0,0.0,"{'avg': 0.39710540949620154, 'normalized_produ...",1.0
